# Specific Heat in Diamond and Lead

We assume to have a material with $3N$ phonon modes, of Einstein frequency $\omega_E$.


## A. System Partition Function

For some solid of N atoms there will be 3N normal modes. Each normal mode, which can be represented classically as a wave of distortion in the lattice of the material, gives rise to a quantum of sound- a phonon of characteristic frequency $\omega_i$ and energy $\hbar \omega_i$. The quantum state of a solid (at low enough temperatures) can be modelled as a volume containing a gas of non-interacting phonons. The partition function for a mode:

$\displaystyle Q_N (V, T) = \sum_n e^{-\beta E_n}$.


For some state which has $n_i$ phonons of type i (i.e. of characteristic frequency $\omega_i$), the total energy of the state is:

$\displaystyle E\{n_i\} = \sum_{i=1}^{3N}n_i \hbar \omega_i$,

and so:

$\displaystyle Q_N (V, T) = \sum_{i=1}^{3N} e^{-\beta E_i}$.


Since we assume that $N \to \infty$, and using the **important assumption** that for an Einstein model all characteristic frequencies are taken to be equal, we can evaluate the expression with an infinite-term geometric progression to obtain:

$Q_N (V, T) = \frac{1}{1 - \exp({-\beta \hbar\omega_E})}$.

Since we assumed that the phonons are independent, the full partition function can be written as a product of the partition functions of nodes:

$\displaystyle Q = \prod^{3N}_{i = 1} Q_N = Q_N^{3N}$.

Therefore, the partition function for such a system is:

$\text{Q} = \prod\limits^{3N}_{i = 1} \frac{1}{1 - \exp(-\beta\hbar\omega_E)}$.



## B. Average Phonon Occupation Levels

The probability of finding n phonons is:

$\displaystyle P_n = \frac{e^{-\beta E_n}}{\sum^{\infty}_{n=0}e^{-\beta E_n}}$,

where the denominator is $Q_N^{-1}$. The average occupation state is then simply the sum of this the probability of finding n phonons times the number of phonons:

$\displaystyle <n> = \sum^{\infty}_{n=0} P_n \cdot n = 1 - e^{-\beta \hbar \omega_E} \cdot \sum^{\infty}_{n=0}ne^{-\beta \hbar\omega_E n} = 1 - e^{-\beta \hbar \omega_E} \cdot \frac{e^{-\beta \hbar \omega_E}}{(1 - e^{-\beta \hbar \omega_E})^2} = \frac{1}{e^{\beta\hbar\omega_E}-1}$

## C. System Total Energy

Average energy can be computed by summing the average phonon occupation times the respective energy per mode:

$\displaystyle U = \sum^{3N}_{i=1} \hbar \omega_i <n_i> \ = \sum^{3N}_{i=1} \frac{\hbar \omega_E}{e^{\beta \hbar \omega_E} - 1}$

OR:

$\ln Q = - 3N \cdot \ln(1 - e^{-\beta \hbar \omega_E})$

$U = - \frac{\partial \ln Q}{\partial \beta} = 3N \frac{1}{1 - e^{-\beta \hbar \omega_E}} \cdot \hbar \omega_E e^{-\beta \hbar \omega_E} = 3N \frac{\hbar \omega_E}{e^{\beta \hbar \omega_E} - 1}$

which both give the same result!

## D. System Heat Capacity ($C_V$)

Heat capacity is the change in energy with respect to temperature. For a solid:

$C_V \approx C_p = C = \frac{\partial U}{\partial T}$

Since $\beta = \frac{1}{k_B T}$:

$C_V = 3N\hbar \omega_E \left( -\frac{\hbar \omega_E}{k_B} (-1) \frac{1}{T^2} \frac{e^{\beta \hbar \omega_E}}{(e^{\beta \hbar \omega_E -1})^{2}} \right) = 3N \hbar^2 \omega_E^2 \beta^2 k_B \left( \frac{e^{\beta \hbar \omega_E}}{(e^{\beta \hbar \omega_E} - 1)^2}\right)$



In [4]:
# Imports

import numpy as np
import scipy as sp
import requests
import pandas as pd
from io import StringIO
import pint #I shall certainly be importing a pint after this assignment is done :)

# !pip install pint


In [5]:
# Data Gathering for smooth sailing

diamond_url = 'https://raw.githubusercontent.com/diodeamy/StatMech/refs/heads/main/data/diamond.dat'
lead_url = 'https://raw.githubusercontent.com/diodeamy/StatMech/refs/heads/main/data/lead.dat'

diamond_resp = requests.get(diamond_url)
diamond_str = diamond_resp.text
diamond_io = StringIO(diamond_str)
diamond_df = pd.read_csv(diamond_io, comment='#', header=None, delim_whitespace=True)

lead_resp = requests.get(lead_url)
lead_str = lead_resp.text
lead_io = StringIO(lead_str)
lead_df = pd.read_csv(lead_io, header=None, delim_whitespace=True)


In [25]:
diamond_df.columns = ['Temperature', 'Specific_Heat']
lead_df.columns = ['Temperature', 'Specific_Heat']

ts_diamond = diamond_df['Temperature']
shs_diamond = diamond_df['Specific_Heat']

ts_lead = lead_df['Temperature']
shs_lead = lead_df['Specific_Heat']

In [ ]:
# k_B = 1.380649e-23     # Boltzmann constant (J/K)
# hbar = 1.0545718e-34   # Reduced Planck's constant (J·s)
# N = 1                 # Number of atoms (can be absorbed into the amplitude if unknown)

# # Einstein model function for fitting
# def einstein_model(T, theta_E):
#     beta = 1 / (k_B * T)
#     x = theta_E / T  # since theta_E = hbar*omega_E / k_B
#     exp_term = np.exp(-x)
#     Cv = 3 * N * k_B * (x**2) * (exp_term / (1 - exp_term)**2)
#     return Cv

# # Fit the function
# popt, pcov = curve_fit(einstein_model, T_data, C_data, p0=[100])  # Initial guess for theta_E in K
# theta_E_fit = popt[0]

# print(f"Fitted Einstein Temperature (Theta_E): {theta_E_fit:.2f} K")

# # Plotting
# T_fit = np.linspace(min(T_data), max(T_data), 300)
# C_fit = einstein_model(T_fit, theta_E_fit)

# plt.plot(T_data, C_data, 'o', label='Experimental Data')
# plt.plot(T_fit, C_fit, '-', label=f'Einstein Fit\n$\\Theta_E$ = {theta_E_fit:.2f} K')
# plt.xlabel('Temperature (K)')
# plt.ylabel('Specific Heat (J/K)')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
def einstein_CV(T, ):

In [19]:
# Data Processing & Misc. for even smoother sailing?
## specific hear diamond data was given in cal/mol/K
ureg = pint.UnitRegistry()

## E. Diamond Einstein Temperature ($T_E$) (by fit)

In this section we look at the datafile for measurements of the specific heat of diamond in order to obtain the Einstein temperature of the system by fitting. 

## F. Diamond Debye Temperature ($T_D$) (by fit)

We wish to find the Debye temperature of diamond, by fitting the Debye model to the measurements of specific heat. We use a formulation such that the solution is more stable for small values of temperature. 

## G. Debye Model vs. Einstein Model Comparison (for diamond)

## H. Debye Model vs. Einstein Model Comparison (for lead)

### H.a. Einstein Temperature ($T_E$) (by fit) - lead

### H.b. Debye Temperature ($T_D$) (by fit) - lead

### H.c. Model Comparison - lead

## I. Diamond vs. Lead Comparison Discussion

## J. Fermi Temperature ($T_F$) (by fit) - lead

## K. Room-Temperature Entropy by Debye Function - diamond & lead